# Step I: Parametric Analysis
This notebook performs a parametric analysis of a building design using EnergyPlus and BESOS helper functions.  

### Various Imports
The .idf model is load, then three different Windows-to-Wall Ratio (WWR) ar set (15%, 50% and 90%). 
The thickness of the isolation material is increased from 0.0 cm up to 0.35 cm.
The output dataframe is saved into the `../data/` directory to be analyzed and plotted from a separate python script.

In [1]:
import matplotlib.pyplot as plt
from seaborn import heatmap
from seaborn import pairplot
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_rows', None)

for fname in os.listdir('../model/idf/9.0/'):
    # File names management
    name, orientation, nat_vent = fname.replace('.idf', '').split('_')
    title = name
    if orientation == '1':
        title += 'N'
    elif orientation == '2':
        title += 'E'
    if nat_vent == '1':
        title += 'on'
    elif nat_vent == '0':
        title += 'off'

    print(f'Processing {title}...')
    
    # Load the building from the IDF file
    building = ef.get_building(f'../model/idf/9.0/{fname}')

    # Define the problem
    insulation = FieldSelector(
        class_name='Material', 
        object_name='Rock wool - unbonded_01', 
        field_name='Thickness'
    )
    glazing1 = FieldSelector(
        class_name='WindowMaterial:Glazing',
        object_name='2',
        field_name='Thickness'
    )
    glazing2 = FieldSelector(
        class_name='WindowMaterial:Glazing',
        object_name='40',
        field_name='Thickness'
    )
    
    #==========================================
    #PARAMETERS
    #==========================================
    # Insulation Thickness: from 0cm to 35cm
    # WWR: [15%, 50%, 90%]
    parameters = [
        Parameter(
            selector=insulation,
            #value_descriptor = RangeParameter(0.03,0.36),
            value_descriptor = RangeParameter(0.0,0.4),
            name='Insulation Thickness'
        ),
        wwr(CategoryParameter(options=[.15,.5,.9])),
        Parameter(
            selector=glazing1,
            value_descriptor = RangeParameter(.007, .05),
            name='Glazing1'
        ),
        Parameter(
            selector=glazing2,
            value_descriptor = RangeParameter(.007, .05),
            name='Glazing2'
        )
    ]

    #================================
    #OBJECTIVES AND PROBLEM
    #================================
    # Define the problem's objectives
    objectives = [
        'Electricity:Facility',
        'DistrictHeating:Facility',
        'DistrictCooling:Facility'
    ]

    problem=EPProblem(parameters, objectives)

    
    samples = pd.DataFrame({
        'Insulation Thickness':[],
        'Window to Wall Ratio':[]
    })

    t_range = []
    w_range = []
    g1_range = []
    g2_range = []
    
    for i in np.linspace(.05, .35, 7):
        for j in [.15, .5, .9]:
            for k in np.linspace(.007, .05, 3):
                t_range.append(i)
                w_range.append(j)
                g1_range.append(k)
                g2_range.append(k)
    
    samples['Insulation Thickness'] = t_range
    samples['Window to Wall Ratio'] = w_range
    samples['Glazing1'] = g1_range
    samples['Glazing2'] = g2_range
    
    # Set the evaluator
    evaluator = EvaluatorEP(
        problem, 
        building,
        '../model/epw/RUS_Moscow.276120_IWEC.epw',
        out_dir='outputdir', 
        err_dir='errdir',
    )

    print('Performing evaluation...')
    outputs = evaluator.df_apply(samples, keep_input=True)
    # Outputs in kWh are obtained by output/3.6e6

    # Set the objectives in kWh
    outputs['Electricity:Facility']/=3.6e6
    outputs['DistrictHeating:Facility']/=3.6e6
    outputs['DistrictCooling:Facility']/=3.6e6
    outputs['TotalConsumption']=outputs['DistrictHeating:Facility']+outputs['DistrictCooling:Facility']
    
    outputs.to_csv(f'../data/eval/{title}.csv')
    print('File Saved.')

Processing modelEon...


/usr/local/lib/python3.7/dist-packages/besos/parameters.py:400: UserWarning: wwr is intended to be used with RangeParameter. Your value_descriptor is CategoryParameter(options=[0.15, 0.5, 0.9])
  warnings.warn(f'wwr is intended to be used with RangeParameter. Your value_descriptor is {value_descriptor}')


Performing evaluation...
File Saved.
Processing modelEoff...


/usr/local/lib/python3.7/dist-packages/besos/parameters.py:400: UserWarning: wwr is intended to be used with RangeParameter. Your value_descriptor is CategoryParameter(options=[0.15, 0.5, 0.9])
  warnings.warn(f'wwr is intended to be used with RangeParameter. Your value_descriptor is {value_descriptor}')


Performing evaluation...
File Saved.
Processing modelNoff...


/usr/local/lib/python3.7/dist-packages/besos/parameters.py:400: UserWarning: wwr is intended to be used with RangeParameter. Your value_descriptor is CategoryParameter(options=[0.15, 0.5, 0.9])
  warnings.warn(f'wwr is intended to be used with RangeParameter. Your value_descriptor is {value_descriptor}')


Performing evaluation...
File Saved.
Processing modelNon...


/usr/local/lib/python3.7/dist-packages/besos/parameters.py:400: UserWarning: wwr is intended to be used with RangeParameter. Your value_descriptor is CategoryParameter(options=[0.15, 0.5, 0.9])
  warnings.warn(f'wwr is intended to be used with RangeParameter. Your value_descriptor is {value_descriptor}')


Performing evaluation...
File Saved.
